In [4]:
import PySimpleGUIWeb as sg
from random import randint
import numpy as np
from time import sleep
from IPython.display import display
import ipywidgets as widgets
from utils import Board

In [5]:
width = widgets.IntSlider(min = 10, max = 50, description = 'Width')
height = widgets.IntSlider(min = 10, max = 50, description = 'Height')
display(width, height)

IntSlider(value=10, description='Width', max=50, min=10)

IntSlider(value=10, description='Height', max=50, min=10)

In [3]:
# layout code from PySimpleGUI cookbook found at:
# https://pysimplegui.readthedocs.io/en/latest/cookbook/#minesweeper-style-grid-of-buttons

board = [[randint(0,1) for j in range(width.value)] for i in range(height.value)]
layout =  [[sg.Button('?', size=(4, 2), key=(i,j), pad=(0,0)) for j 
            in range(width.value)] for i in range(height.value)]
window = sg.Window('Minesweeper', layout)

while True:
    event, values = window.read()
    if event in (None, 'Exit'):
        break
    # window[(row, col)].update('New text')   # To change a button's text, use this pattern
    # For this example, change the text of the button to the board's value and turn color black
    window[event].update(board[event[0]][event[1]], button_color=('white','black'))
window.close()

app closing
server stopped
Returned from Remi Start command... now sending None event


In [ ]:
while True:
    h = height.value
    w = width.value
    board = Board(w, h)
    game_lost = False
    previous_display = board.display.copy()
    exited = False
    mines = board.num_mines
    mode_on = ('black','green')
    mode_off = ('black','red')
    clear_mode = True

    button_grid = [[sg.Button('?', size=(4, 2), key=(i,j),
                             pad=(0,0)) for j in range(w)] for i in range(h)]
    mode_toggle = [[sg.Text("Click Mode: "),
                  sg.Button('Mine', button_color = mode_off, key = 'mine_mode', pad = (0,0)),
                  sg.Button('Clear', button_color = mode_on, key = 'clear_mode', pad = (0,0))]]
    top_row =  [sg.Text("Mines Remaining:"), sg.Text(f"{mines}", pad = (50, 0),
                                                     text_color = 'red',
                                                     background_color='686463',
                                                    key = 'remaining'),
               sg.Column(mode_toggle)]
    layout = [top_row, *button_grid]
    window = sg.Window('Minesweeper', layout)

    while True:
        event, values = window.read()
        if event in (None, 'Exit'):
            exited = True
            break

        if event in ('mine_mode', 'clear_mode'):
            clear_mode = event == 'clear_mode'
            window.FindElement('mine_mode').update(button_color = mode_off if clear_mode else mode_on)
            window.FindElement('clear_mode').update(button_color = mode_on if clear_mode else mode_off)        
            continue

        row, col = event[:2]
        
        if previous_display[row, col]:
            continue

        if not board.update_square(not clear_mode, row, col):
            game_lost = True
            display_diff = np.argwhere(board.display == board.display)
        else:
            mines -= not clear_mode
            if mines == 0:
                break
            display_diff = np.argwhere(previous_display != board.display)
            previous_display = board.display.copy()
            window.FindElement('remaining').update(mines)

        for x, y in display_diff:
            cell_val = board.board[x, y]

            if cell_val == -1:
                cell_val = '*'
                color_update = ('black','grey')
            elif cell_val == 0:
                cell_val = ' '
                color_update = ('white','black')
            else:
                color_update = ('white','black')

            window.FindElement((x, y)).update(cell_val, button_color=color_update)

        if game_lost:
            sleep(3)
            break

    if exited:
        break
            
    result = 'NURTZ!' if game_lost else 'HUZZAH!'
    if sg.PopupYesNo(result + '\nPlay again?') == 'No':
        break
window.close()

app closing
server stopped
Returned from Remi Start command... now sending None event
